In [2]:
import datasets
import daft

daft.set_execution_config(enable_native_executor=True, default_morsel_size=256)
ds = datasets.load_dataset("detection-datasets/fashionpedia")
df_train = daft.from_arrow(ds["train"].data.table[:1000])

fashionpedia_num_classes = ds["train"].features["objects"].feature["category"].num_classes
hf_img_to_daft_img = daft.col("image").struct.get("bytes").image.decode()

df_train = df_train.with_column(
    "image", hf_img_to_daft_img
)
df_train.show(2)

╭──────────┬──────────────┬───────┬────────┬───────────────────────────────────────────────────────────╮
│ image_id ┆ image        ┆ width ┆ height ┆ objects                                                   │
│ ---      ┆ ---          ┆ ---   ┆ ---    ┆ ---                                                       │
│ Int64    ┆ Image[MIXED] ┆ Int64 ┆ Int64  ┆ Struct[bbox_id: List[Int64], category: List[Int64], bbox: │
│          ┆              ┆       ┆        ┆ List[FixedSizeList[Float64; 4]], area: List[Int64]]       │
╞══════════╪══════════════╪═══════╪════════╪═══════════════════════════════════════════════════════════╡
│ 23       ┆ <Image>      ┆ 682   ┆ 1024   ┆ {bbox_id: [150311, 150312, 15…                            │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 25       ┆ <Image>      ┆ 683   ┆ 1024   ┆ {bbox_id: [158953, 158954, 15…                            │
╰──────────┴──────────────┴───────┴────────┴───────────────────────────────────────────────────────────╯

(Showing first 2 rows)

In [4]:
import torch


def apply_torch_transform(bboxes):
    bboxes = torch.tensor(bboxes, dtype=torch.float32)
    return bboxes


df_train = df_train.with_columns(
    {
        "bboxes": daft.col("objects")
        .struct.get("bbox")
        .apply(
            apply_torch_transform,
            return_dtype=daft.DataType.tensor(daft.DataType.float32()),
        ),
        "img_tensor": daft.col("image").cast(daft.DataType.tensor(dtype=daft.DataType.uint8()))
    }
)
df_train.show(2)

╭──────────┬──────────────┬───────┬────────────┬───────────────────────┬───────────────────────┬───────────────────────╮
│ image_id ┆ image        ┆ width ┆      …     ┆ objects               ┆ bboxes                ┆ img_tensor            │
│ ---      ┆ ---          ┆ ---   ┆            ┆ ---                   ┆ ---                   ┆ ---                   │
│ Int64    ┆ Image[MIXED] ┆ Int64 ┆ (1 hidden) ┆ Struct[bbox_id:       ┆ Tensor(Float32)       ┆ Tensor(UInt8)         │
│          ┆              ┆       ┆            ┆ List[Int64],          ┆                       ┆                       │
│          ┆              ┆       ┆            ┆ category:             ┆                       ┆                       │
│          ┆              ┆       ┆            ┆ List[Int64], bbox:    ┆                       ┆                       │
│          ┆              ┆       ┆            ┆ List[FixedSizeList[Fl ┆                       ┆                       │
│          ┆              ┆       ┆            ┆ oat64; 4]], area:     ┆                       ┆                       │
│          ┆              ┆       ┆            ┆ List[Int64]]          ┆                       ┆                       │
╞══════════╪══════════════╪═══════╪════════════╪═══════════════════════╪═══════════════════════╪═══════════════════════╡
│ 23       ┆ <Image>      ┆ 682   ┆ …          ┆ {bbox_id: [150311,    ┆ <Tensor shape=(4, 4)> ┆ <Tensor shape=(1024,  │
│          ┆              ┆       ┆            ┆ 150312, 15…           ┆                       ┆ 682, 3)>              │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 25       ┆ <Image>      ┆ 683   ┆ …          ┆ {bbox_id: [158953,    ┆ <Tensor shape=(10,    ┆ <Tensor shape=(1024,  │
│          ┆              ┆       ┆            ┆ 158954, 15…           ┆ 4)>                   ┆ 683, 3)>              │
╰──────────┴──────────────┴───────┴────────────┴───────────────────────┴───────────────────────┴───────────────────────╯

(Showing first 2 rows)

In [ ]:
sample = df_train[["bboxes", "img_tensor"]].limit(1).to_pydict()

In [ ]:
import albumentations as A

transforms = A.Compose([
    A.RandomResizedCrop(size=(224, 224), antialias=True),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
],
bbox_params=A.BboxParams(format="pascal_voc", label_fields=["category_id"]))
transforms(**sample)

/var/folders/mz/q18tmv191p571kqg_6sn8skh0000gn/T/ipykernel_72438/1286868144.py:4: UserWarning: Argument 'antialias' is not valid and will be ignored.
  A.RandomResizedCrop(size=(224, 224), antialias=True),


ValueError: Your 'label_fields' are not valid - them must have same names as params in dict

In [ ]:

from daft import DataType as dt

@daft.udf(return_dtype=dt.tensor(dt.float32))
def transform(image, bbox):
    out = transforms(image=image, bbox=bbox)
    print(out)
    return out["image"]
df_train[0]
# df_train.with_columns(transform(daft.col("img_tensor"), daft.col("bboxes")))